In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('dataset/train.csv')

In [3]:
df['text']

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object

In [4]:
# create a function that will extract all the unique words from the dataset
def unique_words(df):
    unique_words = []
    for i in range(len(df)):
        for word in df['text'][i].split():
            if word not in unique_words:
                unique_words.append(word)
    return unique_words


In [5]:
arr = unique_words(df)

In [6]:
len(arr)

31924

In [7]:
# create a function that will create a dictionary of all the unique words and their frequency in the dataset
def word_freq(df):
    unique_words = []
    for i in range(len(df)):
        for word in df['text'][i].split():
            if word not in unique_words:
                unique_words.append(word)
    word_freq = {}
    for word in unique_words:
        word_freq[word] = 0
    for i in range(len(df)):
        for word in df['text'][i].split():
            word_freq[word] += 1
    return word_freq

In [8]:
word_dict = word_freq(df)

In [9]:
# do tokenization of all the words present in the dictionary
def tokenize_words(dictionary):
    tokens = {}
    ind = 0
    for word in dictionary:
        tokens[word] = ind
        ind += 1
    return tokens

In [10]:
tokens = tokenize_words(word_dict)

In [11]:
from transformers import BertTokenizerFast
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments

d:\Anaconda\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
